# top

In [1]:
import os, sys
import pickle
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
import sklearn.metrics
import sklearn.linear_model

sys.path.append('..')
import diefComp1Utils as util


/Users/arianprabowo/anaconda3/envs/dief3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folderPath = '/Users/arianprabowo/Library/CloudStorage/OneDrive-UNSW/root/DIEF/raw data/e04_splits/'
pathTrainX = folderPath+'train_X.zip'
pathTrainY = folderPath+'train_Y.csv'
pathTestX = folderPath+'test_X.zip'
pathTestY = folderPath+'test_Y.csv'

# train

In [3]:
dfTrainY = pd.read_csv(pathTrainY, index_col=0)
aTrny = dfTrainY.iloc[:,3:].values ==1

In [4]:
aTrnx = util.load_GlobalFeatureSet1(pathTrainX)
aTrnx.shape

  0%|          | 0/31851 [00:00<?, ?it/s]/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:164: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,2] = scipy.stats.skew(idata['v'])
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:165: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,3] = scipy.stats.kurtosis(idata['v'])
100%|██████████| 31851/31851 [00:31<00:00, 1015.81it/s]


(31851, 11)

In [5]:
l_lr = [] # list of logistic regressors
for ii in tqdm(range(aTrny.shape[1])):
    ilr = sklearn.linear_model.LogisticRegression()
    if (aTrny[:,ii] == 1).sum() == 0:
        l_lr.append(0)
        continue
    ilr.fit(aTrnx,aTrny[:,ii])
    l_lr.append(ilr)

100%|██████████| 240/240 [00:18<00:00, 13.16it/s]


In [7]:
aTrnh = np.zeros((aTrnx.shape[0], aTrny.shape[1]))
for ii in tqdm(range(aTrnx.shape[1])):
    if l_lr[ii] != 0:
        aTrnh[:,ii] = l_lr[ii].predict(aTrnx)
d = util.allMetrics(dfTrainY.values[:,3:], aTrnh)
dfM, dfWM = util.weighted_metrics(d, dfTrainY)
dfWM

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:00<00:00, 203.35it/s]
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:307: RuntimeWarning: invalid value encountered in divide
  precision = tp / (tp + fp)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:308: RuntimeWarning: invalid value encountered in divide
  recall = tp / (tp + fn)
mAP:   0%|          | 0/11 [00:00<?, ?it/s]/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:263: RuntimeWarning: invalid value encountered in divide
  a_precision[ii,:] = true__positive / (true__positive + false_positive)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:264: RuntimeWarning: invalid value encountered in divide
  a_recall[ii,:] = true__positive / (true__positive + false_negative)
mAP: 100%|██████████| 11/11 [00:02<00:00,  4.73it/s]

  Accuracy  | Precision  |   Recall   |     F1     |    mAP     
   0.9879288          0.0          0.0          0.0          0.0


accuracy     0.857736
precision    0.000000
recall       0.000000
f1           0.000000
AP           0.000000
dtype: float64

# test

## load

In [8]:
dfTestY = pd.read_csv(pathTestY, index_col=0)
aTsty = dfTestY.iloc[:,3:].values ==1

In [9]:
aTstx = util.load_GlobalFeatureSet1(pathTestX)

  0%|          | 0/329439 [00:00<?, ?it/s]/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:164: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,2] = scipy.stats.skew(idata['v'])
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:165: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,3] = scipy.stats.kurtosis(idata['v'])
100%|██████████| 329439/329439 [04:59<00:00, 1099.56it/s]


## predict

In [10]:
aTsth = np.zeros((aTstx.shape[0], aTsty.shape[1]))
for ii in tqdm(range(aTstx.shape[1])):
    if l_lr[ii] != 0:
        aTsth[:,ii] = l_lr[ii].predict(aTstx)

100%|██████████| 11/11 [00:00<00:00, 57.81it/s]


In [11]:
dlb = util.parition_wrapper_for_metrics(dfTestY, aTsth, 'leaderboard')
dsc = util.parition_wrapper_for_metrics(dfTestY, aTsth, 'secret')
dcm = util.parition_wrapper_for_metrics(dfTestY, aTsth, 'combined')

/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:307: RuntimeWarning: invalid value encountered in divide
  precision = tp / (tp + fp)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:308: RuntimeWarning: invalid value encountered in divide
  recall = tp / (tp + fn)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:309: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * (precision * recall) / (precision + recall)
mAP:   0%|          | 0/11 [00:00<?, ?it/s]/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:263: RuntimeWarning: invalid value encountered in divide
  a_precision[ii,:] = true__positive / (true__positive + false_positive)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e07_LR/../diefComp1Utils.py:264: RuntimeWarning: invalid value encountered in divide
  a_recall[ii,:] = 

  Accuracy  | Precision  |   Recall   |     F1     |    mAP     
   0.9882037          0.0          0.0          0.0          0.0


mAP: 100%|██████████| 11/11 [00:11<00:00,  1.02s/it]


  Accuracy  | Precision  |   Recall   |     F1     |    mAP     
    0.988086          0.0          0.0          0.0          0.0


mAP: 100%|██████████| 11/11 [00:22<00:00,  2.06s/it]

  Accuracy  | Precision  |   Recall   |     F1     |    mAP     
   0.9881453          0.0          0.0          0.0          0.0


In [12]:
dfM, dfWM = util.weighted_metrics(dcm, dfTestY)
dfWM

accuracy     0.84836
precision    0.00000
recall       0.00000
f1           0.00000
AP           0.00000
dtype: float64

# bot

In [13]:
for ik,iv in dcm.items():
    print(ik, iv.mean())

tp 0.0
tn 322017.4708333333
fp 0.020833333333333332
fn 3877.241666666667
accuracy 0.988145261407229
precision 0.0
recall 0.0
f1 0.0
AP 0.0
_AP_precision 0.0
_AP_recall 0.07692307692307693
